<a href="https://colab.research.google.com/github/napoles-uach/max_distance/blob/main/Random_ClosePack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/napoles-uach/max_distance.git

In [2]:
%cd max_distance/

/content/max_distance


In [3]:
!pip -q install py3Dmol
import py3Dmol

In [4]:
from utils.molecule_functions import Molecule,read_sdf_from_string,read_sdf_from_file,apply_rotation,calculate_contact


In [6]:
molecule_data = read_sdf_from_file("Conformer3D_COMPOUND_CID_4733.sdf")
molecule = Molecule(
    coordinates=[data[:3] for data in molecule_data],
    symbols=[data[3] for data in molecule_data],
    atom_radii={'C': 1.7, 'H': 1.2, 'O': 1.52, 'N': 1.55, 'S': 1.8}
)


In [7]:
import numpy as np
from utils.molecule_functions import Molecule, apply_rotation, calculate_contact

a = np.array([1, 0, 0])
b = np.array([0, 1, 0])
c = np.array([0, 0, 1])
vectors = [a,b,c,b-c,b-a,c-a,a+b,a+c,b+c,a+b+c,a+c-b,a+b-c,b+c-a]

# Set up parameters for the random exploration
num_iterations = 2000  # Number of random samples to try
np.random.seed(42)  # Seed for reproducibility
lista=[]
distancias=[]
a_=[]
for __ in range(num_iterations):
    ang_x, ang_y, ang_z = np.random.randint(0, 360, size=3)
    rotated_molecule = apply_rotation(molecule, (ang_x, ang_y, ang_z))
    global_max_displacement = 0
    parametro_red=[]
    _=[]
    _a_=[]
    i=0
    for latt_vec in vectors:
      max_displacement = calculate_contact(rotated_molecule, latt_vec)
      #print(i,max_displacement)
      if i<3:
        a=max_displacement
      elif i<9:
        a=1/np.sqrt(2)*max_displacement
      else:
        a=1/np.sqrt(3)*max_displacement
      parametro_red.append(a)
                #print(i,max_displacement,a)
      _.append([i,max_displacement])
      _a_.append([i,a])
      i+=1

    parametro_a=max(parametro_red)
    lista.append([ang_x,ang_y,ang_z,parametro_a])
    distancias.append(_)
    a_.append(_a_)

In [8]:
min_indice = min((sublista[3], i) for i, sublista in enumerate(lista))

In [ ]:
lista[min_indice[1]]

In [10]:
def generate_xyz_data(molecule):
    xyz_data = ""
    for i in range(len(molecule.coordinates)):
        x, y, z = molecule.coordinates[i]
        atom_type = molecule.symbols[i]
        xyz_data += f"{atom_type} {x:.3f} {y:.3f} {z:.3f}\n"
    return str(len(molecule.coordinates)) + "\n\n" + xyz_data

In [11]:
ang_x=lista[min_indice[1]][0]
ang_y=lista[min_indice[1]][1]
ang_z=lista[min_indice[1]][2]
rotated_molecule = apply_rotation(molecule, (ang_x, ang_y, ang_z))

In [ ]:
direction_vector = lista[min_indice[1]][3]*vectors[12]
#max_displacement = calculate_contact(rotated_molecule, direction_vector)
displacement_vector = direction_vector#/np.linalg.norm(direction_vector)
#rotated_molecule = apply_rotation(molecule, (min_angles[0], min_angles[1],min_angles[2]))
transformed_coords = rotated_molecule.coordinates + displacement_vector
transformed_molecule = Molecule(transformed_coords, rotated_molecule.symbols, rotated_molecule.atom_radii)

original_xyz = generate_xyz_data(rotated_molecule)
transformed_xyz_a = generate_xyz_data(transformed_molecule)

xyzview = py3Dmol.view(width=800, height=400)
xyzview.addModel(original_xyz, 'xyz')
xyzview.addModel(transformed_xyz_a, 'xyz')
#xyzview.addModel(transformed_xyz_b, 'xyz')
#xyzview.addModel(transformed_xyz_c, 'xyz')
xyzview.setStyle({'sphere': {}})
xyzview.setBackgroundColor('white')
xyzview.zoomTo()
xyzview.show()


In [ ]:
lista_ordenada = sorted(lista, key=lambda x: x[3])
lista_ordenada[:30]

In [14]:
contador = 1
for ang in lista_ordenada:
   a_x,a_y,a_z, dist_i = ang
   rota = apply_rotation(molecule, (a_x, a_y, a_z))
   coords_xyz = generate_xyz_data(rota)
   nombre_archivo = f"{dist_i}.xyz"
   with open(nombre_archivo, 'w') as file:
    file.write(f"{coords_xyz}\n")
   contador+=1

In [15]:
!mkdir moleculas

In [16]:
! mv *.xyz moleculas/

In [ ]:
!tar -czvf moleculas.tgz moleculas